In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import mxnet as mx

from joblib import Parallel, delayed
import time

In [2]:
# labels =  ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']

labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

label_dict = dict([(l,i) for i, l in enumerate(labels)])

# label_dict = dict([(l,i) for i, l in enumerate(labels)] + \
#                   zip(other_labels, range(len(labels), len(labels)+len(other_labels))))

In [3]:
patches_rootdir = '/home/yuncong/CSHL_data_patches/'
model_dir = '/home/yuncong/mxnet_models/'

In [4]:
batch_size = 32

In [ ]:
# model_name='Inception_BN'
# model_iteration = 39
# init_model = mx.model.FeedForward.load(os.path.join(model_dir, model_name), 
#                                      model_iteration, 
#                                      ctx=mx.gpu(),
#                                     numpy_batch_size=batch_size)

# arg_params = init_model.arg_params.copy()
# arg_params.pop('fc_bias');
# arg_params.pop('fc_weight');

In [5]:
model_name='inception-stage1'
model_iteration = 10
init_model = mx.model.FeedForward.load(os.path.join(model_dir, model_name), 
                                     model_iteration, 
                                     ctx=mx.gpu(),
                                    numpy_batch_size=batch_size)


arg_params = init_model.arg_params.copy()
arg_params.pop('fullc_bias');
arg_params.pop('fullc_weight');

In [6]:
n_class = len(labels)
internals = init_model.symbol.get_internals()
output = internals['flatten_output']
fc = mx.symbol.FullyConnected(data=output, name='fullc', num_hidden=n_class)
softmax = mx.symbol.SoftmaxOutput(data=fc, name='softmax')

In [7]:
n_epoch = 10
l_rate = 0.001
# checkpoint_prefix = os.path.join(model_dir, 'experiment0227')
checkpoint_prefix = os.path.join(model_dir, 'experiment0317')

In [8]:
opt = mx.optimizer.SGD(learning_rate=l_rate)

net = mx.model.FeedForward(ctx=mx.gpu(), 
#                            symbol=init_model.symbol, 
                           symbol=softmax, 
                           num_epoch=n_epoch, optimizer=opt,
                           arg_params = arg_params, 
                           aux_params = init_model.aux_params,
                           allow_extra_params = True,
                           numpy_batch_size = batch_size)

In [9]:
mean_img = mx.nd.load(os.path.join(model_dir, 'mean_224.nd'))['mean_img'].asnumpy()

In [25]:
from IPython.html.widgets import FloatProgress
from IPython.display import display

progress_bar = FloatProgress(min=0, max=np.inf)

In [26]:
patches_allClasses = []
patchLabels_allClasses = []

In [27]:
for stack in ['MD589']:

    dm = DataManager(stack=stack)
    stack_has_annotation = True

    if stack_has_annotation:
        table_filepath = os.path.join(patches_rootdir, '%(stack)s_indices_allLandmarks_allSections.h5'%{'stack':stack})
        indices_allLandmarks_allSections = pd.read_hdf(table_filepath, 'indices_allLandmarks_allSections')
        grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')

    first_bs_sec, last_bs_sec = section_range_lookup[stack]
    first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

    progress_bar.min = first_detect_sec
    progress_bar.max = last_detect_sec
    display(progress_bar)
    
    for sec in range(first_detect_sec, last_detect_sec+1):
        
        if sec not in indices_allLandmarks_allSections.columns:
            continue

        progress_bar.value = sec
#         print sec

        ## define grid, generate patches

    #     t = time.time()
        dm.set_slice(sec)
        dm._load_image(['rgb-jpg'])

        patch_size, stride, w, h = grid_parameters.tolist()
        half_size = patch_size/2

        ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                         indexing='xy')

        sample_locations = np.c_[xs.flat, ys.flat]


        q = indices_allLandmarks_allSections[sec].dropna()
        if len(q.index) == 0:
            continue

        for label_ind, label in enumerate(labels):

            print label

            if label == 'BackG':
                w = [q[l] for l in q.index if l.endswith('surround')]
                if len(w) == 0:
                    continue
                else:
                    indices_roi = np.concatenate(w)
            else:
                if label in q.index:
                    indices_roi = q[label]
                else:
                    continue

            n2 = len(indices_roi)
            print n2, 'samples'

            num_sample_each_polygon = 30
            indices_roi = np.random.choice(indices_roi, min(num_sample_each_polygon, n2), replace=False)

            n = len(indices_roi)
            print n, 'used samples'

            sample_locations_roi = sample_locations[indices_roi]

            patches2 = np.asarray([dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
                                  for x, y in sample_locations_roi])

            patches = np.rollaxis(patches2, 3, 1)
            patches = patches - mean_img

            patches_allClasses.append(patches.copy())

            patch_labels = label_ind * np.ones((n, ), np.int)
            patchLabels_allClasses.append(patch_labels)

            del patches, patches2, sample_locations_roi

        del sample_locations

    #     break
    #     sys.stderr.write('generating patches: %.2f seconds\n' % (time.time() - t))


BackG
1662 samples
30 used samples
5N
121 samples
30 used samples
7n
48 samples
30 used samples
7N
12N
Pn
VLL
83 samples
30 used samples
6N
Amb
R
Tz
RtTg
LRt
LC
AP
sp5
BackG
4185 samples
30 used samples
5N
72 samples
30 used samples
7n
49 samples
30 used samples
7N
12N
Pn
VLL
116 samples
30 used samples
6N
Amb
R
Tz
RtTg
LRt
LC
AP
sp5
315 samples
30 used samples
BackG
1907 samples
30 used samples
5N
122 samples
30 used samples
7n
68 samples
30 used samples
7N
29 samples
29 used samples
12N
Pn
VLL
7 samples
7 used samples
6N
Amb
R
Tz
RtTg
LRt
LC
AP
sp5
BackG
4166 samples
30 used samples
5N
231 samples
30 used samples
7n
46 samples
30 used samples
7N
90 samples
30 used samples
12N
Pn
VLL
20 samples
20 used samples
6N
Amb
R
Tz
RtTg
LRt
LC
AP
sp5
159 samples
30 used samples
BackG
5355 samples
30 used samples
5N
198 samples
30 used samples
7n
65 samples
30 used samples
7N
80 samples
30 used samples
12N
Pn
VLL
31 samples
30 used samples
6N
Amb
R
Tz
RtTg
LRt
LC
AP
sp5
175 samples
30 used sampl

In [28]:
patches_allClasses_arr = np.concatenate(patches_allClasses)
patchLabels_allClasses_arr = np.concatenate(patchLabels_allClasses)

In [29]:
len(patchLabels_allClasses_arr)

28054

In [30]:
patches_allClasses_eval = []
patchLabels_allClasses_eval = []

In [31]:
for stack in ['MD594']:

    dm = DataManager(stack=stack)
    stack_has_annotation = True

    if stack_has_annotation:
        table_filepath = os.path.join(patches_rootdir, '%(stack)s_indices_allLandmarks_allSections.h5'%{'stack':stack})
        indices_allLandmarks_allSections = pd.read_hdf(table_filepath, 'indices_allLandmarks_allSections')
        grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')

    first_bs_sec, last_bs_sec = section_range_lookup[stack]
    first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

    progress_bar.min = first_detect_sec
    progress_bar.max = last_detect_sec
    display(progress_bar)
    
    for sec in range(first_detect_sec, last_detect_sec+1):
        
        if sec not in indices_allLandmarks_allSections.columns:
            continue

        progress_bar.value = sec
#         print sec

        ## define grid, generate patches

    #     t = time.time()
        dm.set_slice(sec)
        dm._load_image(['rgb-jpg'])

        patch_size, stride, w, h = grid_parameters.tolist()
        half_size = patch_size/2

        ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                         indexing='xy')

        sample_locations = np.c_[xs.flat, ys.flat]


        q = indices_allLandmarks_allSections[sec].dropna()
        if len(q.index) == 0:
            continue

        for label_ind, label in enumerate(labels):

            print label

            if label == 'BackG':
                w = [q[l] for l in q.index if l.endswith('surround')]
                if len(w) == 0:
                    continue
                else:
                    indices_roi = np.concatenate(w)
            else:
                if label in q.index:
                    indices_roi = q[label]
                else:
                    continue

            n2 = len(indices_roi)
            print n2, 'samples'

            num_sample_each_polygon = 30
            indices_roi = np.random.choice(indices_roi, min(num_sample_each_polygon, n2), replace=False)

            n = len(indices_roi)
            print n, 'used samples'

            sample_locations_roi = sample_locations[indices_roi]

            patches2 = np.asarray([dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
                                  for x, y in sample_locations_roi])

            patches = np.rollaxis(patches2, 3, 1)
            patches = patches - mean_img

            patches_allClasses_eval.append(patches.copy())

            patch_labels = label_ind * np.ones((n, ), np.int)
            patchLabels_allClasses_eval.append(patch_labels)

            del patches, patches2, sample_locations_roi

        del sample_locations

    #     break
    #     sys.stderr.write('generating patches: %.2f seconds\n' % (time.time() - t))


BackG
5687 samples
30 used samples
5N
131 samples
30 used samples
7n
7N
12N
Pn
VLL
66 samples
30 used samples
6N
Amb
R
Tz
RtTg
LRt
LC
AP
sp5
703 samples
30 used samples
BackG
4554 samples
30 used samples
5N
172 samples
30 used samples
7n
7N
12N
Pn
VLL
142 samples
30 used samples
6N
Amb
R
Tz
RtTg
LRt
LC
AP
sp5
470 samples
30 used samples
BackG
4334 samples
30 used samples
5N
292 samples
30 used samples
7n
7N
12N
Pn
VLL
186 samples
30 used samples
6N
Amb
R
Tz
RtTg
LRt
LC
AP
sp5
339 samples
30 used samples
BackG
4557 samples
30 used samples
5N
397 samples
30 used samples
7n
7N
12N
Pn
VLL
56 samples
30 used samples
6N
Amb
R
Tz
RtTg
LRt
LC
AP
sp5
385 samples
30 used samples
BackG
4849 samples
30 used samples
5N
215 samples
30 used samples
7n
63 samples
30 used samples
7N
60 samples
30 used samples
12N
Pn
VLL
256 samples
30 used samples
6N
Amb
R
Tz
RtTg
LRt
LC
AP
sp5
199 samples
30 used samples
BackG
5280 samples
30 used samples
5N
531 samples
30 used samples
7n
58 samples
30 used samples
7N

In [ ]:
len(patches_allClasses_eval)

In [32]:
patches_allClasses_eval_arr = np.concatenate(patches_allClasses_eval)
patchLabels_allClasses_eval_arr = np.concatenate(patchLabels_allClasses_eval)

In [33]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [34]:
batch_size = 32

train_iter = mx.io.NDArrayIter(
    patches_allClasses_arr, 
    patchLabels_allClasses_arr,
    batch_size = batch_size,
    shuffle=True
)

eval_iter = mx.io.NDArrayIter(
    patches_allClasses_eval_arr, 
    patchLabels_allClasses_eval_arr,
    batch_size = batch_size,
    shuffle=True
)

t = time.time()

net.fit(train_iter, eval_data=eval_iter,
        batch_end_callback=mx.callback.Speedometer(batch_size, 30),
        epoch_end_callback=mx.callback.do_checkpoint(checkpoint_prefix))

net.save(checkpoint_prefix)

print time.time() - t

# 100 samples/sec


INFO:root:Start training with [gpu(0)]
INFO:root:Epoch[0] Batch [30]	Speed: 83.86 samples/sec	Train-accuracy=0.804167
INFO:root:Epoch[0] Batch [60]	Speed: 80.32 samples/sec	Train-accuracy=0.773958
INFO:root:Epoch[0] Batch [90]	Speed: 79.76 samples/sec	Train-accuracy=0.768403
INFO:root:Epoch[0] Batch [120]	Speed: 72.89 samples/sec	Train-accuracy=0.775260
INFO:root:Epoch[0] Batch [150]	Speed: 70.37 samples/sec	Train-accuracy=0.775833
INFO:root:Epoch[0] Batch [180]	Speed: 69.87 samples/sec	Train-accuracy=0.774479
INFO:root:Epoch[0] Batch [210]	Speed: 69.36 samples/sec	Train-accuracy=0.772768
INFO:root:Epoch[0] Batch [240]	Speed: 68.22 samples/sec	Train-accuracy=0.777344
INFO:root:Epoch[0] Batch [270]	Speed: 72.55 samples/sec	Train-accuracy=0.776968
INFO:root:Epoch[0] Batch [300]	Speed: 70.20 samples/sec	Train-accuracy=0.779062
INFO:root:Epoch[0] Batch [330]	Speed: 69.81 samples/sec	Train-accuracy=0.779356
INFO:root:Epoch[0] Batch [360]	Speed: 68.74 samples/sec	Train-accuracy=0.781250
INFO

4628.76457715


In [ ]:
q = np.where(patchLabels_allClasses == label_dict['7N'])[0]

for p in patches_allClasses[q]:
    plt.imshow(np.rollaxis(p + mean_img, 0, 3).astype(np.uint8));
    plt.show();